In [46]:
import psycopg2

In [47]:
conn = psycopg2.connect(host="localhost", database="postgres", user="postgres", password="root")
conn.autocommit = True


In [48]:
cursor = conn.cursor()
cursor.execute("select * from inventory")
results = cursor.fetchall()
for row in results:
    print(row)

In [49]:
cursor = conn.cursor()
cursor.execute("""
SELECT stores.store_id,
       100.0 * SUM(sales.units) / (SUM(sales.units) + SUM(inventory.stock_on_hand)) AS sales_proportion
FROM sales
JOIN stores ON sales.store_id = stores.store_id
JOIN inventory ON stores.store_id = inventory.store_id AND sales.product_id = inventory.product_id
GROUP BY stores.store_id;
""")
sales_proportions = cursor.fetchall()

print("Proportion of sales per inventory:")
for store_id, proportion in sales_proportions:
    print(f"Store name: {store_name}, Proportion: {proportion:.2f}%")
    
    




Proportion of sales per inventory:


In [77]:
cursor = conn.cursor()

cursor.execute("""
SELECT sales.year ,
       sales.quarter ,
       SUM(sales.units * products.price) AS total_sales
FROM sales
JOIN products ON sales.product_id = products.product_id
GROUP BY sales.year, sales.quarter;

""")
sales_per_quarter_per_year = cursor.fetchall()

# Print results with formatting
print("Sales per quarter per year:")
for year, quarter, total_sales in sales_per_quarter_per_year:
    # Convert quarter number to human-readable format
    quarter_name = ["Q1", "Q2", "Q3", "Q4"][quarter - 1]
    print(f"Year: {year}, Quarter: {quarter_name}, Total Sales: {total_sales}")


UndefinedColumn: column products.price does not exist
LINE 4:        SUM(sales.units * products.price) AS total_sales
                                 ^


In [76]:
cursor = conn.cursor()

cursor.execute("""
SELECT stores.store_name, SUM(product.price * sales.units) AS revenue
FROM sales
JOIN products ON sales.product_id = products.product_id
JOIN stores ON sales.store_id = stores.store_id
GROUP BY stores.store_name;


""")
revenue_per_store = cursor.fetchall()
print("Revenue per store:")
for store_id, revenue in revenue_per_store:
    print(f"Store ID: {store_id}, Revenue: {revenue}")


UndefinedTable: missing FROM-clause entry for table "product"
LINE 2: SELECT stores.store_name, SUM(product.price * sales.units) A...
                                      ^


In [74]:
cursor = conn.cursor()

cursor.execute("""
SELECT SUM(products.price * sales.quantity) AS revenue
FROM sales
JOIN products ON sales.product_id = products.product_id
GROUP BY products.category;

""")
revenue_per_category = cursor.fetchall()
print("\nRevenue per product category:")
for category, revenue in revenue_per_category:
    print(f"Category: {category}, Revenue: {revenue}")


UndefinedColumn: column products.price does not exist
LINE 2: SELECT SUM(products.price * sales.quantity) AS revenue
                   ^


In [69]:
cursor = conn.cursor()

cursor.execute("""
SELECT  SUM(product.price * sales.quantity) AS revenue
FROM sales
JOIN products ON sales.product_id = products.product_id
GROUP BY products.category;

""")
revenue_per_category = cursor.fetchall()
print("\nRevenue per product category:")
for category, revenue in revenue_per_category:
    print(f"Category: {category}, Revenue: {revenue}")


UndefinedTable: missing FROM-clause entry for table "product"
LINE 2: SELECT  SUM(product.price * sales.quantity) AS revenue
                    ^


In [67]:
cursor = conn.cursor()

cursor.execute("""
SELECT products.product_name, SUM(sales.units) AS total_sold
FROM sales 
JOIN products  ON sales.product_id = products.product_id
GROUP BY  products.product_name
ORDER BY total_sold DESC 
LIMIT 10;
""")
top_10_products = cursor.fetchall()

print("Top 10 sold products:")
for name, total_sold in top_10_products:
    print(f"{name}: {total_sold}")


Top 10 sold products:


In [78]:
cursor = conn.cursor()

cursor.execute("""
SELECT  SUM(sales.quantity * products.profit) AS profit
FROM sales,products 
where sales.product_id = products.product_id
GROUP BY products.category;

""")
profit_per_store = cursor.fetchall()
print("Profit per store:")
for store_id, profit in profit_per_store:
    print(f"Store ID: {store_id}, Profit: {profit}")


UndefinedColumn: column sales.quantity does not exist
LINE 2: SELECT  SUM(sales.quantity * products.profit) AS profit
                    ^


In [79]:

cursor.close()
conn.close()
